In [103]:
import pandas as pd
import numpy as np
from pathlib import Path

# Project root: run notebook with cwd = scripts/analysis (e.g. open from that folder)
PROJECT_ROOT = Path("../..").resolve()
PISA_DIR = PROJECT_ROOT / "PISA"
OUT_DIR = PROJECT_ROOT / "data" / "processed" / "pisa_analysis"

In [122]:
# Load ART+PISA and PISA-only CSVs (paths relative to project root)
AP_file = PISA_DIR / "ART+PISA_2.0" / "CSV" / "ART_PISA_2.0.csv"
P_file = PISA_DIR / "PISA_2.0" / "CSV" / "PISA_2.0.csv"
AP_data = pd.read_csv(AP_file, sep=',')
P_data = pd.read_csv(P_file, sep=',')

In [123]:
#Извлечем названия нужных столбцов
nickname1 = ''
nickname2 = ''
birth = ''

dcolumns = []

for i in AP_data:
    if "ПОМОГИТЕ" in i:
        nickname1 = i
    if "Пожалуйста, снова" in i:
        nickname2 = i
    if "рождения" in i:
        birth = i
    dcolumns.append(i)

print(nickname1)
print(nickname2)
print(birth)


ПОМОГИТЕ!!!11111Пожалуйста, придумайте псевдоним! ВНИМАНИЕ! Сохраните его! ИСПОЛЬЗУЙТЕ во втором тесте! Только так я смогу сравнить ваши результаты в двух опросах. https://i.postimg.cc/TwW2KJ90/st-small-845x845-pad-1000x1000-f8f8f8.jpg
Пожалуйста, снова укажите ваш псевдоним!
Пожалуйста, укажите вашу дату рождения.


In [124]:
#извлечем никнеймы из 2 столбцов из файла PISA
names1 = []
names2 = []

for i in range(len(P_data)):
    names1.append(P_data[nickname1][i])
    names2.append(P_data[nickname2][i])

In [126]:
#form a new dataset
output = pd.DataFrame(columns=dcolumns)
children = pd.DataFrame(columns=dcolumns)
no_birthdate = pd.DataFrame(columns=dcolumns)

r_count = len(AP_data) #длина таблицы
print(len(AP_data), len(P_data)) #сравним длины двух таблиц

for i in range(r_count-1):
    tempA = AP_data.iloc[i+1]
    if (tempA[nickname1] in names1) or (tempA[nickname2] in names2): #если subject представлен в обеих таблицах
        year = str(tempA[birth])
        if len(year) > 4: #если год указан корректно
            if (2022 - int(year[-4:]) > 18): #записываем совершеннолетних
                output = output.append(AP_data.iloc[i+1], ignore_index=True) #add a row; iloc[0] starts with actual data, without a header
            else: #иначе - записываем несовершеннолетних
                children = children.append(AP_data.iloc[i+1], ignore_index=True) #add a row; iloc[0] starts with actual data, without a header
        else: #если года рождения нет - записываем в отдельный файл
            no_birthdate = no_birthdate.append(AP_data.iloc[i+1], ignore_index=True) #add a row; iloc[0] starts with actual data, without a header



323 203


/var/folders/63/bs2t6x0j25s1wrgwd4kwyzzr0000gn/T/ipykernel_9909/1593816103.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(AP_data.iloc[i+1], ignore_index=True) #add a row; iloc[0] starts with actual data, without a header
/var/folders/63/bs2t6x0j25s1wrgwd4kwyzzr0000gn/T/ipykernel_9909/1593816103.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(AP_data.iloc[i+1], ignore_index=True) #add a row; iloc[0] starts with actual data, without a header
/var/folders/63/bs2t6x0j25s1wrgwd4kwyzzr0000gn/T/ipykernel_9909/1593816103.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(AP_data.iloc[i+1], ignore_index=True) #add a row; iloc[0] starts with actual dat

In [129]:
print(len(output))
print(len(children))
print(len(no_birthdate))

153
8
0


In [128]:
# Write outputs to data/processed/pisa_analysis/
OUT_DIR.mkdir(parents=True, exist_ok=True)
pd.DataFrame.from_dict(output).to_csv(OUT_DIR / "adults_dataset2.csv", sep=',', mode='w', index=False)
pd.DataFrame.from_dict(children).to_csv(OUT_DIR / "children_dataset2.csv", sep=',', mode='w', index=False)
pd.DataFrame.from_dict(no_birthdate).to_csv(OUT_DIR / "no_birthdate_dataset2.csv", sep=',', mode='w', index=False)